<a href="https://colab.research.google.com/github/Hamza1122/Pose_Estimation_Yolo_Points/blob/main/testing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from sklearn import preprocessing

category = ['sami', 'sajid', 'amman', 'wajid', 'ahmed', 'shumail', 'steve', 'hannan']


In [ ]:
#!unzip new_test.zip

In [ ]:
data=r"/content/new_test/"
IMG_SIZE = 50

In [ ]:
training_data = []
temp_array=[]
cate =['Unknown_class']
def create_training_data():
    for category in cate:
        
        path = os.path.join(data,category)
        class_num = cate.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                temp_array.append(img_array)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [ ]:
import pandas as pd
import numpy as np

X_test=[]
y_test=[]


df_angle = pd.read_csv('new_angles.csv',names=['images','angles','distance'])
df_angle = df_angle.fillna(0)


df_test=pd.read_csv('new_pose.csv')


df_angle['path'] = df_test['Image']
df=pd.merge(df_test,df_angle, how='left', left_on='Image', right_on='path')   #merge both datasets and duplicates will be removed automatically
df=df.fillna(0)

In [ ]:
df=df.drop(['Image','Class','images','path'],axis=1)

for features, label in training_data:
  X_test.append(features)
  y_test.append(label)

y_test=np.array(y_test)
len(y_test)

153

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()   #normalizing the data
df = min_max_scaler.fit_transform(df)

In [ ]:
from keras.utils import np_utils

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
X_test=X_test.astype('float')
X_test=X_test/ 255 

y_test = np_utils.to_categorical(y_test)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy


json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
y_new = loaded_model.predict([df,X_test])

In [ ]:
results = np.argmax(y_new,axis=1)
results
